# Agent-Based Modelling tutorial block

Notebook created by [Alejandra Ramirez](https://github.com/MA-Ramirez)

## Agent-based modelling

Simulation where autonomous agents react to their environment and interact with each other given a predefined set of rules. 

These rules are formulated based on explicit statements rather than mathematical equations (such as: “If condition X is fulfilled, do action Y, and then perform operation Z on all nearby agents”).
                                    *Datseris & Parlitz. (2022)*

**Advantages**
* It captures emergent phenomena from agent interactions
* Provides natural/intuitive description of the system
* Flexible, very easy to change the rules of the simulation

## [Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/)
Julia framework for agent-based modelling.

Simple to learn and use, yet extendable, focusing on fast and scalable model creation and evolution.

It allows you to focus on the composition of the ABM and not on the implementation details.

To set up an ABM simulation in Agents.jl, a user only needs to follow these steps:

1. **Choose** the kind of **space** that the agents will live in.

2. **Define** the **agent type** (or types, for mixed models) of the ABM.

3. **Define** the **model** within an Agents.jl simulation by using its universal structure `AgentBasedModel`.

4. Provide the **time evolution functions** of the ABM.

5. **Visualize** the model and animate its time evolution.

6. **Collect data**.


In this tutorial block we will model the predator-prey dynamics typically described by the Lokta-Volterra equations.

## Predator-prey dynamics

It describes the dynamics of biological systems in which two species interact, one as a predator and the other as a prey. 

### Lokta-Volterra equations
Typically, the Lokta-Volterra equations are used to model the dynamics, which consist on a pair of first-order nonlinear differential equations.

$$\frac{dx}{dt} = \alpha x - \beta xy$$
$$\frac{dy}{dt} = \delta xy - \gamma y $$

* x is the amount of prey (rabbits) 🐇
* y is the amount of predator (foxes) 🦊
* $dx/dt$ and $dy/dt$ are the instantaneous growth rates of the populations
* $\alpha$, $\beta$, $\gamma$ and $\delta$ are positive real parameters that describe the interaction between species

#### Prey dynamics 🐇
* The prey exhibits an exponential growth without being subject to predation $\alpha x$. 🐇🐇
* The rate of predation is proportional to the rate at which the predators and prey meet $- \beta xy$. 🐇🦊☠️

🐇 The rate of change of the prey's population is proportional to its size, minus the rate at which it is preyed upon. 

#### Predator dynamics 🦊
* The growth of the population is proportional to the predation of the prey $\delta xy$. 🦊🐇🍽
* There is an exponential decay in the abscence of prey. The loss rate of the predators can be due to natural death or emigration $- \gamma y $. 🦊☠️

🦊 The rate of change of the predator's population depends on the rate at which it consumes the prey, minus its intrinsic death rate.

**We will simulate the population dynamics of two species (predator 🦊 and prey 🐇) who live in a common ecosystem and compete over limited resources 🌱.**
*As implemented in the [examples of the Agents.jl](https://juliadynamics.github.io/Agents.jl/stable/examples/predator_prey/)*

### Definition of the predator-prey model

* The environment is a 2D grid.
* It contains Foxes 🦊 (agent type), Rabbits 🐇 (agent type) and Grass 🌱 (spatial property).
* Foxes 🦊 **---** eat **--->** Rabbits 🐇 **---** eat **--->** Grass 🌱

The population will oscillate over time if the correct balance is achieved. If there is no balance, a population may become extinct.

* The **Grass** 🌱 is the **spatial property**. It is a replenishing resource that occupies every position in the grid space.
    - `fully_grown`: the grass can only be consumed if it is fully-grown. It is represent by a `boolean`.
    - `regrowth_time`: once the grass has been consumed, it replenishes after a given amount of time.
    - `countdown`: tracks the delay between being consumed and the regrowth time. ?

* **Foxes** 🦊 and **Rabbits** 🐇 are **agent types** that have identical properties, but different behaviours.
    - `energy`: represents an animal current energy level. If the energy drops below 0, the agent will die.
    - `delta_energy`: controls how much energy is acquired after consuming a food source.
    - `reproduction_prob`: reproduction probability. In this model individuals reproduce asexually *(assumption)*.
    
### Set up of the ABM simulation
0. Load the packages that we will be using

In [1]:
using PkgVersion, Agents, Random

PkgVersion.Version(Agents)

v"5.6.5"

1. **Choose** the kind of **space** that the agents will live in.

* The environment is a 2D grid.

`Agents.GridSpace`represents a grid with dimensionality `dims`. The keyword argument `periodic` denotes if the grid should be periodic on its ends (if the an agent goes through the boundary it will re-appear in the opposite side of the grid with the same properties).

Agents live in a 2D grid with a Chebyshev metric (there are 8 neighbours around each grid point - typical grid setting).

In [2]:
dims = (20, 20)
space = GridSpace(dims, periodic = true)

GridSpace with size (20, 20), metric=chebyshev, periodic=true

* The **Grass** 🌱 is the **spatial property**. It is a replenishing resource that occupies every position in the grid space.
    - `fully_grown`: the grass can only be consumed if it is fully-grown. It is represent by a `boolean`.
    - `regrowth_time`: once the grass has been consumed, it replenishes after a given amount of time. (This is a static parameter).
    - `countdown`: tracks the delay between being consumed and the regrowth time.

In [3]:
# The properties of the Grass will be added as additional model level properties.
#`properties` contains the grass as 2 arrays: whether it is fully grown and the countdown to regrow.
# It also have regrowth_time as a static parameter.

regrowth_time = 30

properties = (
    fully_grown = falses(dims),
    regrowth_time = regrowth_time,
    countdown = zeros(Int, dims),
)

(fully_grown = Bool[0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0], regrowth_time = 30, countdown = [0 0 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0])

2. **Define** the **agent type** (or types, for mixed models) of the ABM. 🦊🐇

* **Foxes** 🦊 and **Rabbits** 🐇 are **agent types** that have identical properties, but different behaviours.
    - `energy`: represents an animal current energy level. If the energy drops below 0, the agent will die.
    - `delta_energy`: controls how much energy is acquired after consuming a food source.
    - `reproduction_prob`: reproduction probability. In this model individuals reproduce asexually *(assumption)*.

An agent type is a standard Julia `mutable struct`. It can be created manually, but typically you'd want to use`@agent`. The types must contain some mandatory fields, which is ensured by using `@agent`. The remaining fields of the agent type are up to user's choice.

The mandatory fields are `id::Int` (which assings a unique identification to each agent) and `pos` (it informs about the position of the agent in the space). You should never directly manipulate the mandatory fields.

We want for our agents to live in a 2D grid, so we will use the minimal agent struct `GridAgent{D}`. - It has an additional `pos::NTuple{D,Int}` (tuple type that contains D elements of type Int).

In [4]:
@agent Fox GridAgent{2} begin
    energy::Float64
    delta_energy::Float64
    reproduction_prob::Float64
end

@agent Rabbit GridAgent{2} begin
    energy::Float64
    delta_energy::Float64
    reproduction_prob::Float64
end

3. **Define** the **model** within an Agents.jl simulation by using its universal structure `AgentBasedModel`.

`AgentBasedModel(AgentType [, space]; properties, kwargs...) → model`

It creates an agent-based model for the given agent type(s) and space.

The agents are stored in a dictionary that maps unique ids (integers) to agents. Use `model[id]` to get the agent with the given `id`.

`properties`: additional model-level properties that can be accesed as `model.properties`.

`scheduler`: it decides the activation order of the agents.

`rng`: It stores the random number generation. It is used in all calls to random functions of the model.

*Note on random number generation:*

Most pseudo-random number generators (PRNGs) are built on algorithms involving some recursive method starting from a base value determined by an input called the "seed". 

The default PRNG in most statistical software is the Mersenne Twister algorithm.
In this particular algorithm, there is a recurrence relation where the input seed establishes the initial set of vectors.
The series of pseudo-random numbers generated by the algorithm is fixed by setting a seed. To set seeds in the generation of random numbers makes our code reproducible.

In [5]:
seed = 23182
rng = MersenneTwister(seed)

model = AgentBasedModel(Union{Rabbit,Fox}, space; properties, rng, scheduler = Schedulers.randomly, warn = false)

AgentBasedModel with 0 agents of type Union{Fox, Rabbit}
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: fully_grown, regrowth_time, countdown

In [6]:
# We now have to populate the model with agents.
# We define a function that will initialize the model and populate it.

function initialize_model(; 
    #KEYWORD ARGUMENTS
    n_rabbits = 100,
    n_foxes = 50,
    #space
    dims = (20,20),
    regrowth_time = 30,
    #agents
    delta_energy_rabbit = 4,
    delta_energy_fox = 20,
    prob_rabbit_reproduce = 0.04,
    prob_fox_reproduce = 0.05,
    #prng
    seed = 23182,
    )
    
    rng = MersenneTwister(seed)
    
    #space
    space = GridSpace(dims, periodic = true)
    properties = (
        fully_grown = falses(dims),
        regrowth_time = regrowth_time,
        countdown = zeros(Int, dims),
    )
    
    #model 
    model = ABM(Union{Rabbit,Fox}, space; 
        properties, rng, scheduler = Schedulers.randomly, warn = false
    )
    
    #Add agents to the model
    for _ in 1:n_rabbits
        energy = rand(model.rng, 1:(delta_energy_rabbit*2)) - 1
        add_agent!(Rabbit, model, energy, delta_energy_rabbit, prob_rabbit_reproduce)
    end   
    for _ in 1:n_foxes
        energy = rand(model.rng, 1:(delta_energy_fox*2)) - 1
        add_agent!(Fox, model, energy, delta_energy_fox, prob_fox_reproduce)
    end
    
    #Add grass to the 2D grid with a random initial growth value
    #positions(model) runs an iterator over all positions of a model with a discrete space.
    for p in positions(model)
        # define
        fully_grown = rand(model.rng, Bool)
        countdown = fully_grown ? regrowth_time : rand(model.rng, 1:regrowth_time) - 1
        
        #set
        # slurping (...) is used to pass the position as if they were multiple positions
        model.fully_grown[p...] = fully_grown
        model.countdown[p...] = countdown
        
    end
    return model
end

foxrabbitgrass = initialize_model()

AgentBasedModel with 150 agents of type Union{Fox, Rabbit}
 space: GridSpace with size (20, 20), metric=chebyshev, periodic=true
 scheduler: randomly
 properties: fully_grown, regrowth_time, countdown

4. Provide the **time evolution functions** of the ABM.

We will now define the behaviour/interactions of 🐇,🦊 and 🌱.

**Behaviour of the 🐇 and 🦊**.
The behaviour of both agent types is similar.
* Both move to a random adjacent position with the `walk!` function.
* Both lose 1 energy unit by moving to an adjacent position.
* Both consume a food source if available.
* If the energy level of an agent is below zero, it dies. Otherwise, it lives and reproduce with some probability.

In Agents.jl, an agent-based model should be accompanied with at least one (and at most two) stepping functions.

An *agent step function* is required by default. It defines what happens to an agent when it activates.
- It must accept two arguments: 1. an agent instance 2. a model instance
- Our `agent_step!`function is called `rabbitwolf_step!` and it is dispatched to the appropriate agent type via Julia's Multiple Dispatch system.

A *model step function* can be also needed sometimes. It changes all agents at once, or changes a model property.
- It must accept one argument: 1. the model


*Note*

We will use the following Agents.jl function to define the time evolution:
* `walk!(agent, rand, model)`: moves the agents in the grid. The function invokes a random walk by providing the `rand` function, when refering to the direction in which the agent moves. `ifempty` will check that the target position is unoccupied and only move if that's true. (Foxes can move into a position where there is a rabbit, in order to eat it.)
* `kill_agent!(agent, model)`: removes an agent from the model.

In [7]:
function rabbitfox_step!(rabbit::Rabbit, model)
    #Both move to a random adjacent position with the walk! function
    walk!(rabbit, rand, model)
    #Both lose 1 energy unit by moving to an adjacent position
    rabbit.energy -= 1
    
    #If the energy level of an agent is below zero, it dies
    if rabbit.energy < 0
        kill_agent!(rabbit, model)
        return
    end
    #Both consume a food source if available.
    eat!(rabbit, model)
    
    #Otherwise, it lives and reproduce with some probability.
    if rand(model.rng) <= rabbit.reproduction_prob
        reproduce!(rabbit, model)
    end  
end

function rabbitfox_step!(fox::Fox, model)
    #move
    walk!(fox, rand, model; ifempty=false)
    fox.energy -= 1
    
    #die
    if fox.energy < 0
        kill_agent!(fox, model)
        return
    end
    
    #eat
    #If there is any rabbit on the same grid position, it is dinner time!
    dinner = first_rabbit_in_position(fox.pos, model)
    if !isnothing(dinner)
        eat!(fox, dinner, model)
    end
    
    #reproduce
    if rand(model.rng) <= fox.reproduction_prob
        reproduce!(fox, model)
    end    
end

rabbitfox_step! (generic function with 2 methods)

In [8]:
#If there is any rabbit on the same grid position, it is dinner time!
function first_rabbit_in_position(pos, model)
    #return the ids of agents in the position corresponding to position
    ids = ids_in_position(pos, model)
    #finds the first agent type Rabbit in the list of agents that are in the position 
    j = findfirst(id -> model[id] isa Rabbit, ids)
    
    #if there is no Rabbit in the position, it returns nothing
    if isnothing(j)
        return nothing
    #if there is a Rabbit in the position, it returns that agent (type Rabbit)
    else
        return model[ids[j]]::Rabbit
    end
end

first_rabbit_in_position (generic function with 1 method)

We will now define the `eat!` functions. Rabbits and foxes have separate `eat!`functions.

Foxes 🦊 **---** eat **--->** Rabbits 🐇 **---** eat **--->** Grass 🌱

* If a fox 🦊 eats a rabbit 🐇, the rabbit dies and the fox acquires additional energy.
* If a rabbit 🐇 eats grass 🌱, the rabbit will acquire additional energy and the grass will not be available for consumption until regrowth time has elapsed.

In [9]:
function eat!(fox::Fox, rabbit::Rabbit, model)
    kill_agent!(rabbit, model)
    fox.energy += fox.delta_energy
    return
end

function eat!(rabbit::Rabbit, model)
    if model.fully_grown[rabbit.pos...]
        rabbit.energy += rabbit.delta_energy
        model.fully_grown[rabbit.pos...] = false
    end
    return
end

eat! (generic function with 2 methods)

We will now define the `reproduce!` function. Rabbits and foxes share the same `reproduce!`function.

* Reproduction has a cost of $\frac{1}{2}$ the current energy level of the parent. 
* The offspring is an exact copy of the parent, with exception of `id`.

*Agent.jl functions that we will use:*
* `nextid(model)`: return a valid `id` for creating a new agent with it.
* `add_agent_pos!(agent, model)`: add the agent to the `model`at the agent's own position.

*To create the offpsring we will take advantage of generic programming*
* Generic programming allows the code to apply to as many situations as possible without requiring any changes to the original code itself.
* Parameterized types: whenever we annotate an argument as a type that is not defined yet or does not exist, we have the capability of using a parameter to determine that type. `{parameter}`
* `where` is used to denote what type, or sub-type is passed through the function. This allows us to create a dispatch, to direct the function based on what sort of type is passed.

In [10]:
function reproduce!(agent::A, model) where {A}
    agent.energy /= 2
    
    id = nextid(model)
    offspring = A(id, agent.pos, agent.energy, agent.reproduction_prob, agent.delta_energy)
    add_agent_pos!(offspring, model)
    
    return
end

reproduce! (generic function with 1 method)

**Behaviour of the 🌱**
* If it is fully grown, it is consumable. Otherwise, it cannot be consumed until it regrows after a delay specified by `regrowth_time`.
* Note: the countdown goes from regrowth_time to 0, when it is 0 it is fully_grown (and the countdown is reset again for when it is eaten).
* The dynamics of the grass is our `model_step!` function.

* Julia uses bounds checking to ensure program safety when accessing arrays. You may wish to skip these bounds checks to improve runtime performance in some cases. `@inbounds` macro tells the compiler to skip such bounds checks within the given block.

In [11]:
function grass_step!(model)
    @inbounds for p in positions(model)
        
        if !(model.fully_grown[p...])
            if model.countdown[p...] <= 0
                model.fully_grown[p...] = true
                model.countdown[p...] = model.regrowth_time
            else
                model.countdown[p...] -=1
            end
        end
        
    end
end

grass_step! (generic function with 1 method)

We will now **Run the model**

In [12]:
rabbit(a) = a isa Rabbit
fox(a) = a isa Fox
count_grass(model) = count(model.fully_grown)

count_grass (generic function with 1 method)

In [13]:
rabbitfoxgrass = initialize_model()
steps = 1000
adata = [(rabbit, count), (fox, count)]
mdata = [count_grass]
adf, mdf = run!(rabbitfoxgrass, rabbitfox_step!, grass_step!, steps; adata, mdata)

(1001×3 DataFrame
  Row │ step   count_rabbit  count_fox 
      │ Int64  Int64         Int64     
──────┼────────────────────────────────
    1 │     0           100         50
    2 │     1            83         52
    3 │     2            75         54
    4 │     3            67         56
    5 │     4            59         66
    6 │     5            50         80
    7 │     6            35         88
    8 │     7            30         91
    9 │     8            27         94
   10 │     9            26         97
   11 │    10            21         96
  ⋮   │   ⋮         ⋮            ⋮
  992 │   991            23          0
  993 │   992            25          0
  994 │   993            20          0
  995 │   994            23          0
  996 │   995            29          0
  997 │   996            35          0
  998 │   997            34          0
  999 │   998            26          0
 1000 │   999            22          0
 1001 │  1000            23          0
        